# Eigenvector centrality on subreddits from Reddit

Author: [picorana](https://picorana.github.io/)

First of all, we import all the libraries that we are going to use.

In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

import networkx as nx

init_notebook_mode(connected=True)

As part of the init setup, we define a set of functions that will help us drawing the graphs we are going to use to show the results:

In [2]:
def loadLayout():
    l = Layout(
        title='<br>Network graph made with Python',
        titlefont=dict(size=16),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        annotations=[ dict(
            text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
            showarrow=False,
            xref="paper", yref="paper",
            x=0.005, y=-0.002 ) ],
        xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False))
    
    return l

edge_trace = Scatter(
    x=[],
    y=[],
    line=Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

node_trace = Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=Marker(
        showscale=True,
        colorscale='Portland',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))


def drawSpringLayoutGraph(test_G):
    
    pos = nx.spring_layout(test_G)
    
    for edge in test_G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]

    for node in test_G.nodes():
        x, y = pos[node]
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        
    node_trace['marker']['size'] = []
    
    for node in test_G.nodes():
        node_trace['marker']['size'].append(20)

    for node, adjacencies in enumerate(test_G.adjacency_list()):
        node_trace['marker']['color'].append(len(adjacencies))
        node_info = '# AAA: '+str(len(adjacencies)) + '\n node number: ' + str(node)
        node_trace['text'].append(node_info)

    fig = Figure(data = Data([edge_trace, node_trace]), layout = loadLayout())

    iplot(fig)

Let us play with a toy graph in the beginning, here we define it:

In [3]:
test_G = nx.Graph()
test_G.add_edge(0, 1)
test_G.add_edge(1, 2)
test_G.add_edge(2, 3)
test_G.add_edge(3, 1)
test_G.add_edge(1, 4)
test_G.add_edge(4, 5)
test_G.add_edge(5, 6)
test_G.add_edge(5, 7)
test_G.add_edge(5, 8)
test_G.add_edge(5, 9)

And here we draw it:

In [4]:
#drawSpringLayoutGraph(test_G)

# Reddit test

Now, let's load our actual data.

In [5]:
G = nx.read_edgelist(open('data/edgelist_weighted_clean.txt', 'rb'))

In [6]:
eigenvector_centrality = nx.eigenvector_centrality(G)

In [7]:
def trim_graph_edges_for_visualization(G):
	for node in G.nodes():
		for edge in G.edges(node):
			if G[edge[0]][edge[1]]['weight'] < 0.045:
				G.remove_edge(edge[0], edge[1])

	solitary= [n for n,d in G.degree_iter() if d==0]
 	G.remove_nodes_from(solitary)
	return G

def drawSpringLayoutGraphFromDict(G, node_sizes):
    
    pos = nx.spring_layout(G)
    
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]

    for node in G.nodes():
        x, y = pos[node]
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        
    node_trace['marker']['size'] = []
    
    for node in G.nodes():
        node_trace['marker']['size'].append(node_sizes[node]*500)
        node_info = 'name: ' + str(node)
        node_trace['text'].append(node_info)

    for node, adjacencies in enumerate(G.adjacency_list()):
        node_trace['marker']['color'].append(len(adjacencies))
        

    fig = Figure(data = Data([edge_trace, node_trace]), layout = loadLayout())

    iplot(fig)

In [8]:
print G.number_of_nodes()
print G.number_of_edges()

6909
518493


In [9]:
G2 = trim_graph_edges_for_visualization(G)

In [10]:
print G2.number_of_nodes()
print G2.number_of_edges()

2026
5912


In [11]:
drawSpringLayoutGraphFromDict(G2, eigenvector_centrality)